In [1]:
import os
import pandas as pd
import numpy as np
import math
import time
import datetime
import networkx as nx
import matplotlib as plt
import matplotlib.pyplot as plt
import random
import ast
import warnings
warnings.filterwarnings('ignore')  #忽略pandas发出的无效警告
from tqdm import tqdm #进度条库

1.逐个节点攻击，获得单一节点失效情况下网络的韧性值，探讨一下韧性值与点度中心性、介数中心性、客流量之间的相关性

In [2]:
#################################################数据集########################################################
#地铁网络表格
network = pd.read_csv(r'J:/IC卡-北京/地铁交通流分配/数据集/subway_network.csv',encoding = 'gbk')
#站点编号
stop_num = pd.read_csv(r'J:/IC卡-北京/地铁交通流分配/数据集/stop_num.csv',encoding = 'gbk')
#客流量与路径表格
OP = pd.read_csv(r'J:/IC卡-北京/地铁交通流分配/数据集/初始路径计算2.0/OD+Initial_Path.csv',encoding='gbk')
OP['Path'] = OP['Path'].apply(ast.literal_eval) #将Path列的数据转换为列表，不然后面会识别不了

<div class="burk">
备注：因为构建网络时需要对换乘节点进行虚拟化，所以在这个会出现一个换乘站点对应多个编号。所以先按照编号逐个攻击
节点，最后再将一个站点对应一个编号的结果进行合并，并归纳到编号数值较小的那一个编号。</div><i class="fa fa-lightbulb-o "></i>

In [3]:
#定义函数，提取相关记录
#值得注意的是，此处的筛选的记录不一定是刚好能在节点失效时段时的碰到的记录
def related_records(df, target_list): #输入的是待处理表格、节点修复顺序列表
    target_list = [str(x) for x in target_list]  # 将目标列表中的元素转换为字符串类型
    # 筛选出 f_timestamp 列中数据小于等于 1557824400 的行
    df1 = df.loc[df['f_timestamp'] < 1557824400] #出发时间点小于时段终点的记录
    # 再筛选出 t_timestamp 列中数据大于等于 1557817200 的行
    df2 = df1.loc[df['t_timestamp'] > 1557817200] #到达时间点大于时间段起点的记录，df2就是符合时间范围的记录
    #接下来筛选是否经过失效站点的记录
    condition = df['Path'].apply(lambda x: any(str(i) in str(x) for i in target_list)) #加一个经过失效站点的条件                                                                     
    result_df = df2[condition] #符合条件的存储为result_df
    result_df1 =  df2.loc[~condition] #不符合条件的存储为result_df1
    return result_df,result_df1 #返回result_df是为了减少后期寻找路径的计算量，返回没被筛选的是计算韧性值时用到

In [4]:
#定义节点失效时段字典生成函数
def generate_dict(target_list, t_start, duration):  #传入失效节点修复序列，起始时间节点，节点修复所需时间
    result_dict = {}
    x = 0
    for num in target_list:
        x = x + 1
        result_dict[num] = (t_start, t_start + x * duration)
    return result_dict 

In [5]:
#定义函数计算到达失效站点的编号、时间、失效站点前一个站点编号、到达失效站点前一个站点时间
def calculate_time(row): #输入的是每一行的表格数据
    node_list = row['Path'] #路径的节点列表
    travel_time = row['Travel_time'] #路径的出行时间
    f_timestamp = row['f_timestamp'] #该出行记录的起始时刻
    total_nodes = len(node_list) #获取路径的节点个数也就是路径长度
    for i, node in enumerate(node_list): #i是该节点在列表中的索引，node就是该节点的编号
        if node in target_list and i != 0:  # 判断该节点是否是失效节点，并且并非是起点，保证可以返回前一个节点
            prev_node = node_list[i-1] #Path返回符合条件节点的前一个节点
#             if prev_node in target_list:  # 判断失效节点的前一个节点是否也是失效节点
#                 continue
            sx_start, sx_end = stop_sx_range[node] #获取节点的失效时间的起终点时刻
            node_arrival = f_timestamp + travel_time*(i/total_nodes) #到达该失效节点的时刻
            node_before_arrival = f_timestamp + travel_time*((i-1)/total_nodes) #到达该失效节点前一个节点的时刻
            if sx_start <= node_arrival <= sx_end:#如果节点到达时刻正好在失效时段
                fail_stop = node #失效节点就是该节点
                un_stop = prev_node #失效节点的前一个节点
                time_fail = int(node_arrival +180) #到达失效节点的时间
                time_before = int(node_before_arrival + 180) #到达失效节点前一个节点的时间
                return pd.Series({'fail_stop': fail_stop, 'un_stop': un_stop, 'time_fail': time_fail, 'time_before': time_before})
        elif node in target_list and i == 0:
            prev_node = node #否则失效节点的前一个节点也是其自身
            sx_start, sx_end = stop_sx_range[node] #获取节点的失效时间的起终点时刻
            node_arrival = f_timestamp
            if sx_start <= node_arrival <= sx_end:
                fail_stop = node #失效节点就是该节点
                un_stop = prev_node #失效节点的前一个节点
                time_fail = f_timestamp
                time_before = f_timestamp
                return pd.Series({'fail_stop': fail_stop, 'un_stop': un_stop, 'time_fail': time_fail, 'time_before': time_before})
    # Return NaN for fail_stop, un_stop, time_fail, and time_before columns if matching node is not found
    return pd.Series({'fail_stop':None, 'un_stop':None, 'time_fail':None, 'time_before':None})

In [6]:
#移除网络中的失效节点
def remove_failed_nodes(network, failed_nodes):
    network_xf = network[~network['Source_num'].isin(failed_nodes) & ~network['Target_num'].isin(failed_nodes)] #~表示取反
    return network_xf #返回去除失效站点的网络

# 计算最短路径的权重之和
def calculate_shortest_path_weight(G, un_stop, target_num):#计算最短路的出行时间,输入的是网络，失效站点的前一个站点，原始终点
    try:
        if un_stop not in G.nodes or target_num not in G.nodes:
            raise ValueError("起点或终点不在网络中")
        path = nx.shortest_path(G, source=un_stop, target=target_num, weight='weight')
        weight_sum = nx.path_weight(G, path, weight='weight') + 300 #加上疏散时间
        return weight_sum
    except (nx.NetworkXNoPath, ValueError) as e:
        return None

def Remaining_path(start_num, end_num, lst):#输入任点1、点2，路径列表，输出点1和点2之间的路径列表
    start_idx = lst.index(start_num)
    end_idx = lst.index(end_num)
    if start_idx > end_idx:
        start_idx, end_idx = end_idx, start_idx
    return lst[start_idx:end_idx+1]
    
# 计算 List 列给定路径的权重之和并扩大
def calculate_list_path_weight(G, un_stop, target_num, list_data):#这是计算换乘公交的剩余路径出行时间
    path = Remaining_path(un_stop,target_num,list_data) #剩余的路径通过上面定的函数来求得
    weight_sum = nx.path_weight(G, path,weight='weight') #计算剩余路径的长度
    return weight_sum * 2.51 #按照公交的出行时间是地铁的出行时间的2.51倍来计算，公交15，地铁44.4

In [7]:
#定义网络生成函数
def creat_network(network): #创建网络生成函数，输入不同的network文件会生成对应的网络，生成初始表格只需要输入初始网络表格数据
    # 创建空的图
    G = nx.Graph()
    # 添加节点和边
    for _, row in network.iterrows():
        src = row['Source_num']
        dst = row['Target_num']
        time = row['Travel_time']
        src_lon = row['Source_lon']
        src_lat = row['Source_lat']
        dst_lon = row['Target_lon']
        dst_lat = row['Target_lat']   
        # 添加节点
        G.add_node(src, pos=(src_lon, src_lat))
        G.add_node(dst, pos=(dst_lon, dst_lat))
        # 添加边
        G.add_edge(src, dst, weight=time)
    # 设置节点位置
    pos = nx.get_node_attributes(G, 'pos')
    # 设置边权重
    labels = nx.get_edge_attributes(G, 'weight')
    return G

In [8]:
#####查找失效站点，并返回失效站点列表
def find_failed_nodes(time_before, dict1):
    failed_nodes = []
    for key, value in dict1.items():
        if value[0] <= time_before <= value[1]:#如果失效节点正好在失效时段之内，那么就返回失效站点编号
            failed_nodes.append(key)
    return failed_nodes

# 移除网络表格中失效节点对应的行，也就是在网络中删除失效节点
def remove_failed_nodes(network,target_list):
    network_xf = network[~network['Source_num'].isin(target_list) & ~network['Target_num'].isin(target_list)] #~表示取反
    return network_xf #返回去除失效站点的网络

# 计算最短路径的权重之和
def calculate_shortest_path_weight(G, un_stop, target_num):#计算最短路的出行时间,输入的是网络，失效站点的前一个站点，原始终点
    try:
        if un_stop not in G.nodes or target_num not in G.nodes:
            raise ValueError("起点或终点不在网络中")
        path = nx.shortest_path(G, source=un_stop, target=target_num, weight='weight')
        weight_sum = nx.path_weight(G, path, weight='weight') + 300 #加上疏散时间
        return weight_sum
    except (nx.NetworkXNoPath, ValueError) as e:
        return None

def Remaining_path(start_num, end_num, lst):#输入任点1、点2，路径列表，输出点1和点2之间的路径列表
    start_idx = lst.index(start_num)
    end_idx = lst.index(end_num)
    if start_idx > end_idx:
        start_idx, end_idx = end_idx, start_idx
    return lst[start_idx:end_idx+1]
    
# 计算 List 列给定路径的权重之和并扩大
def calculate_list_path_weight(G, un_stop, target_num, list_data):#这是计算换乘公交的剩余路径出行时间
    path = Remaining_path(un_stop,target_num,list_data) #剩余的路径通过上面定的函数来求得
    weight_sum = nx.path_weight(G, path,weight='weight') #计算剩余路径的长度
    return weight_sum * 2.51 #按照公交的出行时间是地铁的出行时间的2.51倍来计算，公交15，地铁44.4

In [9]:
def calculate_weight(group):
    first_value = group.iloc[0] #选取每一组中的第一行数据，进行计算
    un_stop = first_value['un_stop']
    target_num = first_value['Target_num']
    list_data = first_value['Path']
    time_before = first_value['time_before']
    failed_nodes  = find_failed_nodes(time_before, stop_sx_range)
    network_sx = remove_failed_nodes(network,failed_nodes)
    G = creat_network(network_sx)
    weight_sum = calculate_shortest_path_weight(G, un_stop, target_num)
    if weight_sum is None:
        G = G0
        weight_sum = calculate_list_path_weight(G, un_stop, target_num, list_data)
    group['time_part2'] = weight_sum #给每个小组的time_part2列赋值
    return group
#调用函数
#Related_records = Related_records.groupby(['Target_num', 'un_stop']).apply(calculate_weight) # 需要传入函数calculate

In [15]:
data = [] #定义一个空列表用于存储计算生成的性能值和其对应的站点编号
t_start = 1557817200 #要计算的时段起点
duration = 720 #计算的周期
G0 = creat_network(network)

for failed_node in range(340,395): #遍历1-395，左闭右开
    target_list = [500,501,502,503,504,505,506,507,508] #在循环里面新建列表，每一个循环清空一次列表，这样使得列表里只有该次遍历的节点
    target_list.append(failed_node) #广义上的失效列表
    
    stop_sx_range = generate_dict(target_list=target_list, t_start=t_start, duration=duration) #调用自定义函数
    Related_records,Unrelated_records = related_records(OP, target_list) #获取相关记录
    if Related_records.empty:
        q = 1
    else:
        len(Related_records),len(Unrelated_records)
        All_size = len(Related_records) + len(Unrelated_records)
    
        Related_records[['fail_stop', 'un_stop', 'time_fail', 'time_before']] = Related_records.apply(calculate_time, axis=1)
        #起点就是失效站点，算作直接换乘公交的群体
        Group_tobus = Related_records[Related_records['fail_stop'] == Related_records['un_stop']]
        time_tobus = Group_tobus['Travel_time'].sum()
        delay_Grouptobus = 1.51*time_tobus #起点就失效的乘客换乘公交产生的延误
    
        Related_records = Related_records[~Related_records['fail_stop'].isna()] #isnull和isna是有区别的，筛选经过失效站点的出行记录用于计算，减少计算量
        Affected_size = len(Related_records) #这里的Related_records已经是筛选出有失效站点记录的表格了
        Group_normal = Related_records[Related_records['fail_stop'].isna()] #不经过失效站点的出行记录，只要计算总出行时间和人数就可以
        #计算df2的参数N
        normal_trip = len(Group_normal)
        normal_time = Group_normal['Travel_time'].sum()
    
        Related_records = Related_records.groupby(['Target_num', 'un_stop']).apply(calculate_weight) # 需要传入函数calculate

        Related_records['time_part1'] = Related_records['time_before'] - Related_records['f_timestamp']
        #计算出等待人群的等待时间
        Related_records['wait_duration'] = Related_records['time_before'].apply(lambda x: 3600 - ((x - 1557817200) % 3600) if ((x - 1557817200) % 3600) >= 3300 else None)
        # 填充缺失值为0，计算更新计算后的出行时间表格
        Related_records['Travel_time_now'] = Related_records['time_part1'].fillna(0) + Related_records['time_part2'].fillna(0) + Related_records['wait_duration'].fillna(0)
        conditions = (Related_records['Travel_time_now'] == 0)
        Related_records.loc[conditions, 'Travel_time_now'] = Related_records.loc[conditions, 'Travel_time']
    
        #计算延误指数和中断指数
        Delay_Index = (Related_records['Travel_time_now'].sum() - Related_records['Travel_time'].sum() + delay_Grouptobus)/(Unrelated_records['Travel_time'].sum() + Related_records['Travel_time'].sum() + normal_time + time_tobus)
        Interruption_Index = Affected_size/All_size
    
        #标准化延误指数和中断指数
        Standard_Delay_Index = Delay_Index/1.51
        Standard_Interruption_Index = Interruption_Index

        #计算性能指标
        a = 0.5 #权重值，取值在0到1之间
        q = 1-(a *  Standard_Delay_Index + ( 1-a )* Standard_Interruption_Index)

    data.append((failed_node, q))
    print(failed_node)

df_Q = pd.DataFrame(data, columns=['stop', 'Q'])

340
341
342
343
344
345
346
347
348
349
350
351
352
353
354
355
356
357
358
359
360
361
362
363
364
365
366
367
368
369
370
371
372
373
374
375
376
377
378
379
380
381
382
383
384
385
386
387
388
389
390
391
392
393
394


In [16]:
df_Q.to_csv('J:/IC卡-北京/地铁交通流分配/数据集/早高峰单点攻击韧性2.csv',encoding = 'gbk', index=False)

2.同一个节点，攻击时长不同，看攻击时长对网络的影响，这里主要变化的量是客流量的变化，可以看看网络韧性与交通流变化之间相关性